In [ ]:
%%local
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
from datetime import datetime
pd.set_option('max_colwidth', None)

In [ ]:
%load_ext kamu

In [ ]:
%import_dataset rocketpool.reth.mint-burn

In [ ]:
%%sql -o reth_pool -n 999999999999999
select 
    block_time, 
    case 
        when event_name = "TokensMinted" then "Mint"
        when event_name = "TokensBurned" then "Burn"
    end as event_name, 
    avg(eth_amount / amount) as ratio 
from `rocketpool.reth.mint-burn` 
group by block_time, event_name
order by 1

In [ ]:
%%local
reth_pool.hvplot.step(
    x="block_time", 
    by="event_name", 
    width=900, height=600, 
    legend='top_left', grid=True, 
    title="ETH : rETH Ratio (Minting and Burning)",
)

In [ ]:
%%local
# TODO: Fit a linear regression to get the growth rate

In [ ]:
%import_dataset coinpaprika.tickers.eth-usd.24h-historic

In [ ]:
%%sql -o eth2usd -n 999999999999
select * from `coinpaprika.tickers.eth-usd.24h-historic` order by event_time

In [ ]:
%%local
eth2usd.hvplot.step(x="event_time", y="price", height=500, width=800)

In [ ]:
%import_dataset account.transactions
%import_dataset account.tokens.transfers

In [ ]:
%%sql
select * from `account.tokens.transfers`

In [ ]:
%%sql -o reth_tx
with 

tx_fpv as (
    select
        *,
        cast(value as float) / power(10.0, 18) as value_fp
    from `account.transactions`
),

tx_deltas as (
    select
        *,
        case
            when to = "0xeadb3840596cabf312f2bc88a4bb0b93a4e1ff5f" then value_fp
            when from = "0xeadb3840596cabf312f2bc88a4bb0b93a4e1ff5f" then -value_fp
            else null
        end as eth_amount
    from tx_fpv
),


tokens_fpv as (
    select
        *,
        cast(value as float) / power(10.0, token_decimal) as value_fp
    from `account.tokens.transfers`
),

tokens_deltas as (
    select
        *,
        case
            when to = "0xeadb3840596cabf312f2bc88a4bb0b93a4e1ff5f" then value_fp
            when from = "0xeadb3840596cabf312f2bc88a4bb0b93a4e1ff5f" then -value_fp
            else null
        end as token_amount
    from tokens_fpv
),

tokens_eth as (
    select
        d.block_time,
        d.block_number,
        d.transaction_hash,
        d.token_symbol,
        d.token_amount,
        t.eth_amount
    from tokens_deltas as d
    left join tx_deltas as t
    on d.transaction_hash = t.transaction_hash
)

select
    block_time, 
    block_number,
    transaction_hash,
    token_amount,
    sum(token_amount) over (order by block_time) as token_balance,
    eth_amount,
    sum(eth_amount) over (order by block_time) as eth_balance
from tokens_eth
where token_symbol = "rETH"
order by 1

In [ ]:
%%local
reth_tx["eth_invested"] = -reth_tx.eth_balance
reth_tx.hvplot.step(x="block_time", y="eth_invested", title="ETH Invested")

In [ ]:
%import_dataset account.tokens.investments
%import_dataset account.tokens.market-value.reth

In [ ]:
%%sql -o investments -n 99999999
select * from `account.tokens.investments`

In [ ]:
%%local
# TODO: group by day?
investments[
    investments.token_symbol == "rETH"
].hvplot.bar(
    x="block_time", 
    y="token_amount", 
    title="rETH Buy/Sell Transactions",
)

In [ ]:
%%sql -o reth_mv
select * from `account.tokens.market-value.reth`

In [ ]:
%%local
reth_mv.hvplot.line(
    x="event_time", 
    y=["token_book_value_eth", "token_market_value_eth"], 
    legend="bottom_right",
    title="rETH: Book vs Market Value in ETH",
    height=500, 
    width=800,
)

In [ ]:
%%local
reth_mv.hvplot.line(
    x="event_time", 
    y=["token_book_value_eth_as_usd", "token_market_value_usd"], 
    legend="bottom_right",
    title="rETH: Book vs Market Value in USD",
    height=500, 
    width=800,
)

In [ ]:
%import_dataset alphavantage.tickers.daily.spy
%import_dataset account.whatif.reth-vs-snp500.transactions
%import_dataset account.whatif.reth-vs-snp500.market-value

In [ ]:
%%sql -o spy_ticks -n 9999999999999
select * from `alphavantage.tickers.daily.spy`

In [ ]:
%%local
spy_ticks.hvplot.step(
    x="event_time", 
    y=["close"],
    title="SPY: Daily Close Price in USD",
    height=500, 
    width=800,
)

In [ ]:
%%sql -o alt_mv -n 99999999999
select * from `account.whatif.reth-vs-snp500.market-value`

In [ ]:
%%local
alt_mv.hvplot.line(
    x="event_time", 
    y=["token_book_value_eth_as_usd", "token_market_value_usd", "alt_spy_market_value_usd"], 
    legend="bottom_right",
    title="rETH vs S&P500",
    height=500, 
    width=800,
)